In [2]:
import requests
import json
import time

from secrets import server_url, token  # edit the secrets.py file and enter your details

In [9]:
# setting times for project

now = time.time()  # time since epoch start in seconds
now_in_ms = now * 1000  # time since epoch start in milliseconds
two_days_in_ms = 60 * 60 * 24 * 2 * 1000 #  60 seconds * 60 minutes * 24 hours * 2 = 48 hours worth of milliseconds

deadline = now_in_ms + two_days_in_ms  # now in milliseconds + 48 hours worth of milliseconds = deadline 48 hours in the future

In [10]:
# this is where we prepare the request

# authentication header - this will remain the same and be used in all subsequent requests
auth_header = {"X-AUTH-ACCESS-TOKEN": token}

#  endpoint
url = server_url + "/projects"

#  data 
data = {
          "customerId": 1,                      # <- This ID is specific to the XTRF instance 
          "sourceLanguageId": 219,              # <- This ID is specific to the XTRF instance 
          "targetLanguagesIds": [               # <- This square bracket starts an "array" 
            73                                  # <- This ID is specific to the XTRF instance
          ],                                    # <- This square bracket closes the array
          "specializationId": 7,                # <- This ID is specific to the XTRF instance
          "serviceId": 8,                       # <- This ID is specific to the XTRF instance
          "dates": {                            # <- This curly brace opens a nested JSON object
            "startDate": {                      # <- This curly brace opens another nested JSON object
              "time": now_in_ms                 # <- so-called epoch time, in milliseconds
            },                                  # <- This curly brace closes the startDate nested JSON object
            "deadline": {
              "time": deadline                  # <- so-called epoch time, in milliseconds
            }
          },                                    # <- This curly brace closes the nested JSON object
          "name": "XTRF Summit 2020 #online",   # <- The name of the project, will be carried over to tasks 
          "people": {
            "responsiblePersons": {
              "projectManagerId": 1             # <- This ID is specific to the XTRF instance 
            },
            "customerContacts": {               # <- customerContacts is optional, if not set, XTRF will add default customer contact
                "primaryId": 1,                 # <- This ID is specific to the XTRF instance
                "sendBackToId": 2               # <- This ID is specific to the XTRF instance
            }
          },
          "instructions": {                     # <- instructions are optional
            "forProvider": "Instructions for linguists <a href='https://google.com'>GO HERE<a>",
            "fromCustomer": "The naming of this field has always puzzled me. :)",
          },
        }

In [11]:
#  here we make the actual request

response = requests.post(url, json=data, headers=auth_header)

# and load the response in order to access the structured data 
loaded_json = json.loads(response.content)

 

In [6]:
# here we get the complete project data, including task data

# project ID
# the project ID is stored as an integer in the JSON response, and is here converted to a string so that it can be concatenated to the URL
project_id = str(loaded_json["id"])  

# endpoint
url = server_url + "/projects/" + project_id

# parameters
parameters = {"embed": "tasks"}

# we make the request
response = requests.get(url, params=parameters, headers=auth_header)

# and then load the response
complete_project_data = json.loads(response.content)



In [7]:
# here we upload the file and get the token from the response

#  endpoint
url = server_url + "/files"

#  file contents
file = {'file': open("source_file.docx", "rb")}

#  file meta data (name)
data = {"fileName": "source_file.docx"}

# we make the request - refer to Requests docs for sending files
# https://requests.readthedocs.io/en/master/user/quickstart/#post-a-multipart-encoded-file
response = requests.post(url, files=file, json=data, headers=auth_header)

# and then load the response
loaded_json = json.loads(response.text)

# from the response we get the file token 
file_token = loaded_json["token"]



In [8]:
# here we add the uploaded file to the first task in the array of tasks in the complete project data 

#  task ID 
task_id = str(complete_project_data["tasks"][0]["id"])  

#  endpoint
url = server_url + "/tasks/" + task_id + "/files/input"

#  data
data = {"token": file_token, 
        "category": "WORKFILE"}

# we make the request
final_response = requests.post(url, json=data, headers=auth_header)
final_response.status_code

204